Run MMVT and open the sample subject with the DKT cortical atlas

In [2]:
from src.mmvt_addon.scripts import run_mmvt
mmvt = run_mmvt.run('sample', 'dkt')

Writing output to /autofs/space/thibault_001/users/npeled/mmvt/sample/logs/run_mmvt.log
"./blender" "/autofs/space/thibault_001/users/npeled/mmvt/sample_dkt.blend"  --python "/local_mount/space/thibault/1/users/npeled/code/mmvt/src/mmvt_addon/scripts/run_mmvt.py" -- --subject "sample" --subjects "" --atlas "dkt" --real_atlas "aparc.DKTatlas40" --bipolar "False" --debug "False" --overwrite "True" --back "True" 
We got the mmvt object (tcp://127.0.0.1:8001)!


Load MEG source estimate from mne-python sample dataset

In [4]:
import os.path as op
import mne

mne_sample_data_fol = mne.datasets.sample.data_path()
meg_subject = 'sample'
stc_fname = op.join(mne_sample_data_fol, 'MEG', 'sample', 'sample_audvis-meg-rh.stc')
stc = mne.read_source_estimate(stc_fname)

Create the Jupyter notebook widgets

In [5]:
import ipywidgets as widgets
from ipywidgets import HBox, VBox, Label, Button, Layout, Image

T = stc.lh_data.shape[1] 
play = widgets.Play(value=0, min=0, max=T, step=1, interval=1000)
time_slider = widgets.IntSlider(value=0, min=0, max=T, step=1)
widgets.jslink((play, 'value'), (time_slider, 'value'))
quality_slider = widgets.IntSlider(value=40, min=0, max=100, step=1, layout=Layout(width='100%'))
zoom_slider = widgets.IntSlider(value=0, min=-5, max=5, step=1, layout=Layout(width='100%'))
threshold_slider = widgets.FloatSlider(value=2, min=0, max=10, step=0.1, layout=Layout(width='20%'))

Smooth the source estimate and plot it in MMVT for t=0

In [6]:
import numpy as np
def create_stc_t(stc, t):
    data = np.concatenate([stc.lh_data[:, t:t + 1], stc.rh_data[:, t:t + 1]])
    vertices = [stc.lh_vertno, stc.rh_vertno]
    stc_t = mne.SourceEstimate(data, vertices, stc.tmin + t * stc.tstep, stc.tstep, subject=subject)
    return stc_t

def plot_stc(stc, t):
    stc_t = create_stc_t(stc, t)
    stc_t_smooth = mne.morph_data(meg_subject, subject, stc_t, n_jobs=n_jobs, grade=vertices_to)
    meg_fname = mmvt.plot_stc_t(
        stc_t_smooth.rh_data, stc_t_smooth.lh_data, t, threshold=threshold_slider.value, save_image=True)
    mmvt.set_current_t(t)
    return meg_fname, stc_t_smooth
    
n_jobs = 4
t = 0 # ms
vertices_to = mne.grade_to_vertices(subject, None)    
meg_fname, stc_t_smooth = plot_stc(stc, t);
file = open(meg_fname, "rb")
meg_image = Image(value=file.read(), format='png', width=800, height=450)

/autofs/space/thibault_001/users/npeled/anaconda3.4.4/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


surface source space inferred...
Computing morph matrix...
    Left-hemisphere map read.
    Right-hemisphere map read.
    Left-hemisphere map read.
    Right-hemisphere map read.


/autofs/space/thibault_001/users/npeled/anaconda3.4.4/lib/python3.6/site-packages/mne/utils.py:877: DeprecationWarning: Function morph_data is deprecated; This function is deprecated and will be removed in version 0.18. Use morph = mne.compute_source_morph(...) and morph(stc)
  warnings.warn(msg, category=DeprecationWarning)


    17 smooth iterations done.
    14 smooth iterations done.
[done]
[done]


Create the buttons widgets

In [7]:
def update_image(fname=''):
    fname = mmvt.save_image('stc', index=time_slider.value) if fname == '' else fname
    file = open(fname, "rb")
    meg_image.value = file.read()


sagital_button = Button(description="Sagital")
sagital_button.on_click(lambda x:(mmvt.show_sagital(), mmvt.zoom(-1), update_image()))
coronal_button = Button(description="Coronal")
coronal_button.on_click(lambda x:(mmvt.show_coronal(), mmvt.zoom(-1), update_image()))
axial_button = Button(description="Axial")
axial_button.on_click(lambda x:(mmvt.show_axial(), mmvt.zoom(-1), update_image()))
zoom_button = Button(description="zoom")
zoom_button.on_click(lambda x:(mmvt.zoom(zoom_slider.value), update_image()))
render_button = Button(description='Render')
update_button = Button(description='Update')
update_button.on_click(lambda x:(update_image()))
render_button.on_click(lambda x:(
    mmvt.hide_subcorticals(), mmvt.set_brain_transparency(1), mmvt.set_to_camera_view(),
    update_image(mmvt.render_image(quality=quality_slider.value))))

In [8]:
def on_value_change(change):
    t = change['new']
    meg_fname, stc_t_smooth = plot_stc(stc, t)
    file = open(meg_fname, "rb")
    meg_image.value = file.read()

time_slider.observe(on_value_change, names='value') 
VBox([HBox([meg_image, VBox(
    [sagital_button, coronal_button, axial_button, update_button,
    Label('Quality:'), quality_slider, render_button])]),
      HBox([play, time_slider, Label('Threshold:'), threshold_slider])])

Widget Javascript not detected.  It may not be installed or enabled properly.


surface source space inferred...
Computing morph matrix...
    Left-hemisphere map read.
    Right-hemisphere map read.
    Left-hemisphere map read.
    Right-hemisphere map read.


/autofs/space/thibault_001/users/npeled/anaconda3.4.4/lib/python3.6/site-packages/mne/utils.py:877: DeprecationWarning: Function morph_data is deprecated; This function is deprecated and will be removed in version 0.18. Use morph = mne.compute_source_morph(...) and morph(stc)
  warnings.warn(msg, category=DeprecationWarning)


    17 smooth iterations done.
    14 smooth iterations done.
[done]
[done]
surface source space inferred...
Computing morph matrix...
    Left-hemisphere map read.
    Right-hemisphere map read.
    Left-hemisphere map read.
    Right-hemisphere map read.


/autofs/space/thibault_001/users/npeled/anaconda3.4.4/lib/python3.6/site-packages/mne/utils.py:877: DeprecationWarning: Function morph_data is deprecated; This function is deprecated and will be removed in version 0.18. Use morph = mne.compute_source_morph(...) and morph(stc)
  warnings.warn(msg, category=DeprecationWarning)


    17 smooth iterations done.
    14 smooth iterations done.
[done]
[done]
surface source space inferred...
Computing morph matrix...
    Left-hemisphere map read.
    Right-hemisphere map read.
    Left-hemisphere map read.
    Right-hemisphere map read.


/autofs/space/thibault_001/users/npeled/anaconda3.4.4/lib/python3.6/site-packages/mne/utils.py:877: DeprecationWarning: Function morph_data is deprecated; This function is deprecated and will be removed in version 0.18. Use morph = mne.compute_source_morph(...) and morph(stc)
  warnings.warn(msg, category=DeprecationWarning)


    17 smooth iterations done.
    14 smooth iterations done.
[done]
[done]
surface source space inferred...
Computing morph matrix...
    Left-hemisphere map read.
    Right-hemisphere map read.
    Left-hemisphere map read.
    Right-hemisphere map read.


/autofs/space/thibault_001/users/npeled/anaconda3.4.4/lib/python3.6/site-packages/mne/utils.py:877: DeprecationWarning: Function morph_data is deprecated; This function is deprecated and will be removed in version 0.18. Use morph = mne.compute_source_morph(...) and morph(stc)
  warnings.warn(msg, category=DeprecationWarning)


    17 smooth iterations done.
    14 smooth iterations done.
[done]
[done]
